In [5]:
import pandas as pd
import numpy as np
import 

In [13]:
train_data = pd.read_json('train.json')

train_data.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [14]:
print "Train data size: {}".format(len(train_data))
print "Types of cuisine: {}".format(len(pd.Series.unique(train_data['cuisine'])))

Train data size: 39774
Types of cuisine: 20
